In [1]:
import pandas as pd

news_articles = pd.read_csv('data/news_articles.csv')

In [6]:
topic = news_articles.iloc[0]["content"]

In [61]:
topic = '''# Northvolt's Bankruptcy: A Setback for Europe's EV Ambitions

In a week that rocked Europe's electric vehicle (EV) industry, the Swedish battery manufacturer Northvolt filed for bankruptcy protection in a New York court. The move, a consequence of a cash flow crisis, has sent ripples through the continent's plans to bolster its battery manufacturing capabilities. Northvolt's troubles come amidst a broader slowdown in EV demand across Europe.

## Financial Struggles and Market Dynamics

Founded in 2016 by CEO Peter Carlsson, Northvolt rapidly gained a reputation as a shining star in Europe's EV battery production. This perception was bolstered by an aggressive expansion, with more than $15 billion raised from a constellation of investors, including Volkswagen, Goldman Sachs, and BMW. Despite these efforts, Northvolt's current financial status is bleak. The company has declared liabilities amounting to 5.8 billion dollars, with only $30 million in liquid assets left.

The unprecedented filing comes as the once-booming EV market begins to wane. Northvolt's restructuring adviser, Scott Millar, noted that the company's business strategies were based on the assumption of sustained growth in the EV sector. Yet, 2023 has seen a decline in sales, exacerbated by economic inflation and consumer hesitancy. Data from Rho Motion indicates a 3 percent decrease in European EV sales, with a particularly steep 18 percent drop in Germany.

## A Blow to European Battery Independence

Northvolt was conceived as a key player in reducing Europe's dependency on Chinese battery manufacturers such as CATL and BYD. The company's expansion into Germany, including plans for a significant factory in Heide, aligned with these strategic goals. However, the insolvency petition threatens to delay the project, now projected to start cell assembly in late 2027 instead of the previously scheduled end of 2026.

The Heide plant delays also cast a shadow over the substantial public funding allocated to the project. The EU Commission had approved over 902 million euros for the factory, with additional support from Germany's federal and Schleswig-Holstein regional governments. Yet no state funds have been dispersed, pending resolution of Northvolt's restructuring.

## Navigating the Road Ahead

Despite the turmoil, Northvolt remains optimistic about its future. The company envisions completing its reorganization by the first quarter of 2025, maintaining operations throughout this period. Volkswagen has reaffirmed its continued collaboration with the troubled firm, while Scania has extended a $100 million loan to sustain battery cell production in Skellefteå, Northvolt's main site.

Nevertheless, the current situation underscores the fragility of Europe's ambitions to cultivate a thriving battery industry. Ambitious startups like Northvolt face myriad challenges, from predicting consumer trends accurately to navigating the complex landscape of industrial finance.

## Conclusion

As Northvolt reorganizes under Chapter 11 protection, the broader European battery industry must reckon with potential repercussions. The EV market is in flux, with consumer demand proving difficult to predict and financial markets growing more risk-averse. The coming months will be crucial for Northvolt and its stakeholders, whose support remains pivotal for a successful turnaround.

The narrative of Northvolt's struggle is a reminder of the volatility innate to high-stakes technological ventures. As Europe endeavors to assert greater autonomy in battery production, the experience of Northvolt may pave the way for refined strategies, mindful of both market dynamics and the critical role of stable financing.'''

In [62]:
# create pydantic class with texts
from pydantic import BaseModel
from typing import List

class Texts(BaseModel):
    text: str

class NewsArticle(BaseModel):
    output: List[Texts]



In [63]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
def generate_voiceover_texts(topic):
    """
    Generate four captivating sentences about a topic using OpenAI's GPT-4.
    
    Args:
        topic (str): The topic to generate sentences about
    """
    client = OpenAI()
    
    prompt = f"""Create 4 captivating, emotionally engaging short sentences about the news below. Requirements:
     - Each sentence MUST be 10-15 words maximum
    - Use active voice and strong verbs
    - Focus on one key point per sentence
    - Avoid complex clauses or conjunctions
    - Include details and numbers where relevant
    - End with a powerful message or call to action
    
    Format the output as 4 separate sentences.
    
    Example flow:
    ["Introduction and hook", "Key benefits or developments", "Practical impact or application", "Future outlook and call to action"]
    
    News:
    {topic}"""

    response = client.beta.chat.completions.parse(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are an expert storyteller and marketing copywriter."},
            {"role": "user", "content": prompt}
        ],
        response_format=NewsArticle
    )
    
    # Split the response into individual sentences
    json_str = response.choices[0].message.parsed
    return json_str.output

In [64]:
texts = generate_voiceover_texts(topic)

In [65]:
voiceover_texts = [text.text for text in texts]
voiceover_texts

["Northvolt's bankruptcy challenges Europe's drive for electric vehicle battery independence.",
 "Their $15 billion investment dreams shattered amid Europe's slowing EV demand by 3%.",
 "Delays in Northvolt's projects threaten Germany's crucial battery manufacturing aspirations.",
 'Europe must confront its dependency on foreign batteries and revise strategic plans.']

In [66]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()

def create_render(texts):
    """
    Create a render using the Creatomate API with a list of voiceover texts.
    
    Args:
        texts (list): List of 4 strings for voiceover texts
    """
    url = "https://api.creatomate.com/v1/renders"
    headers = {
       "Authorization": f"Bearer {os.getenv('CREATOMATE_API_KEY')}",
        "Content-Type": "application/json"
    }
    
    # Define the image URLs
    images = {
        "Image-1": "https://creatomate.com/files/assets/6e448991-32c0-46ff-9043-d0a74fd99b41",
        "Image-2": "https://creatomate.com/files/assets/0f1547eb-647e-429c-a04a-b414d2e2657b",
        "Image-3": "https://creatomate.com/files/assets/08ffb646-af54-4e91-9b38-3eae161fb770",
        "Image-4": "https://creatomate.com/files/assets/e0063a09-ab39-4a41-9c11-359cb339a3db",
    }

    
    # Create modifications dictionary combining images and voiceovers
    modifications = {
        **images,  # Unpack the images dictionary
        **{f"Voiceover-{i+1}": text for i, text in enumerate(texts)}  # Add voiceover texts
    }
    
    payload = {
        "template_id": "16dcc861-b57e-41ac-a83e-ba5623697051",
        "modifications": modifications
    }
    
    response = requests.post(url, headers=headers, json=payload)
    return response.json()

# Make the API call
result = create_render(voiceover_texts)
print(result)

[{'id': 'd6a339c9-b84d-4168-9627-b494915d759f', 'status': 'planned', 'url': 'https://cdn.creatomate.com/renders/d6a339c9-b84d-4168-9627-b494915d759f.mp4', 'snapshot_url': 'https://cdn.creatomate.com/snapshots/d6a339c9-b84d-4168-9627-b494915d759f.jpg', 'template_id': '16dcc861-b57e-41ac-a83e-ba5623697051', 'template_name': 'Short-Form Voice Over', 'template_tags': [], 'output_format': 'mp4', 'modifications': {'Image-1': 'https://creatomate.com/files/assets/6e448991-32c0-46ff-9043-d0a74fd99b41', 'Image-2': 'https://creatomate.com/files/assets/0f1547eb-647e-429c-a04a-b414d2e2657b', 'Image-3': 'https://creatomate.com/files/assets/08ffb646-af54-4e91-9b38-3eae161fb770', 'Image-4': 'https://creatomate.com/files/assets/e0063a09-ab39-4a41-9c11-359cb339a3db', 'Voiceover-1': "Northvolt's bankruptcy challenges Europe's drive for electric vehicle battery independence.", 'Voiceover-2': "Their $15 billion investment dreams shattered amid Europe's slowing EV demand by 3%.", 'Voiceover-3': "Delays in N